In [130]:
import os
import subprocess
import re
import time

# --- Setup Paths (Ensure these environment variables are correctly set in your environment) ---
mopmc_home = os.getenv('MOPMC_HOME')
storm_home = os.getenv('STORM_HOME')
prism_home = os.getenv('PRISM_HOME')

input_folder = os.path.join(os.getcwd(), 'model_size_eval_input')
switch_model_list = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.prism')]
switch_model_20 = os.path.join(input_folder, 'switch_model_20.prism')
switch_property = os.path.join(input_folder, 'switch_prop_achievable.props')

# Construct full executable paths with checks
prism_executable = os.path.join(prism_home, 'bin', 'prism') if prism_home else None
storm_executable = os.path.join(storm_home, 'build', 'bin', 'storm') if storm_home else None
mopmc_executable = os.path.join(mopmc_home, 'build', 'mopmc') if mopmc_home else None

switch_model_list

['/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_20.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_120.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_200.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_100.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_140.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_80.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_60.prism',
 '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_180.prism',
 '/home/guoxin/PycharmProje

In [14]:
#prism_command_swtich_model = f"{prism_executable} {switch_model} {switch_property}"
#! $prism_command_swtich_model
#storm_command_swtich_model = f"{storm_executable} --prism {switch_model} --prop {switch_property}"
#! $storm_command_swtich_model
#mopmc_command_switch_model = f"{mopmc_executable} -m {switch_model} -p {switch_property} -q achievability"
#! $mopmc_command_switch_model

PRISM
=====

Version: 4.4
Date: Thu May 29 15:24:57 AEST 2025
Hostname: guoxin-Precision-3660
Memory limits: cudd=1g, java(heap)=1g
Command line: prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_model_20.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props

Parsing model file "/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_model_20.prism"...

Parsing properties file "/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props"...

1 property:
(1) multi(R{"ctrl_cost"}<=1.895 [ C ], R{"headway_cost"}<=0.573 [ C ], R{"ttc_cost"}<=0.235 [ C ], R{"onlane_cost"}<=0.235 [ C ])

Type:        MDP
Modules:     swtich 
Variables:   loc ctrl conf ct cc headway ttc onlane 

---------------------------------------------------------------------

Model checking: multi(R{"ctrl_cost"}<=1.895 [ C ], R{"headway_cost"}<=0.573 [ C ], R{"ttc_co

In [139]:
# --- Define the parsing function ---
def parse_output(output_string, tool_name, query):
    """
    Parses the output of the PRISM tool and extracts relevant metrics.
    """
    extracted_data = {}
    states_match = re.search(r"States:\s*(\d+)", output_string, re.IGNORECASE)
    transitions_match = re.search(r"Transitions:\s*(\d+)", output_string, re.IGNORECASE)
    choices_match = re.search(r"Choices:\s*(\d+)", output_string, re.IGNORECASE)
    loop_count_match = re.search(r"MOPMC main loop count:\s*(\d+)", output_string, re.IGNORECASE)
    loop_count_key = query + '_loop_count_' + tool_name
    
    if states_match:
        extracted_data['States'] = int(states_match.group(1))
    if choices_match:
        extracted_data['Choices'] = int(choices_match.group(1))
    if transitions_match:
        extracted_data['Transitions'] = int(transitions_match.group(1))
    if loop_count_match:
        extracted_data[loop_count_key] = int(loop_count_match.group(1))
    return extracted_data

In [141]:
def run_tool_and_parse(command_list, parser_function=None, tool_name="Tool", query=None):
    """
    Executes a command and, optionally, parses its output using a provided function.

    Args:
        command_list (list): The command and its arguments as a list.
        tool_name (str): A descriptive name for the tool (e.g., "PRISM", "Storm").
        parser_function (callable, optional): A function that takes the tool's
                                               stdout (string) and returns a dict of extracted data.
                                               If None, no parsing is performed.

    Returns:
        dict: A dictionary containing extracted data and a 'success' flag.
              Returns an empty dictionary if the executable is not found or an error occurs.
    """
    extracted_data = {}

    if not command_list or not command_list[0] or not os.path.exists(command_list[0]):
        print(f"Error: {tool_name} executable not found or command list is invalid. "
              f"Attempted executable: {command_list[0] if command_list else 'None'}")
        return extracted_data

    try:
        print(f"Running {tool_name} command: {' '.join(command_list)}")
        result = subprocess.run(
            command_list,
            capture_output=True,
            text=True,
            check=True
        )
        tool_output = result.stdout
        if parser_function:
            parsed_info = parser_function(tool_output, tool_name, query)
            extracted_data.update(parsed_info) # Add parsed info to the result
    except Exception as e:
        print(f"An unexpected error occurred while running {tool_name}: {e}")

    return extracted_data



In [142]:
def generate_aq_command_list(executable, model_list, prop, tool_name, query='aq'):
    command_list = []
    if query!='aq':
        raise ValueError(f"Unsupported query: {query}!")
    for model in model_list:
        if tool_name == 'mopmc':
            command = [executable, '-m', model, '-p', prop, '-q', 'achievability']            
        elif tool_name == "mopmc_cpu":
            command = [executable, '-m', model, '-p', prop, '-q', 'achievability', '-v', 'standard']
        elif tool_name == "storm":
            command = [executable, '--prism', model, '--prop', prop]
        elif tool_name == "prism":        
            command = [executable, model, prop]
        else:
             raise ValueError(f"Incorrect tool name for achievability query: {tool_name}!")
        command_list.append(command)
    return command_list


def generate_cq_command_list(executable, model_list, prop, tool_name, query='ccq'):
    command_list = []
    
    for model in model_list:
        if query=='ccq':
            if tool_name == 'mopmc':
                command = [executable, '-m', model, '-p', prop, '-q', 'convex']            
            elif tool_name == "mopmc_cpu":
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-v', 'standard']
            else:
                 raise ValueError(f"Incorrect tool name for convex query: {tool_name}!")
        elif query=='ucq':    
            if tool_name == 'mopmc':
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-c', 'n']            
            elif tool_name == "mopmc_cpu":
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-c', 'n', '-v', 'standard']
            else:
                 raise ValueError(f"Incorrect tool name for convex query: {tool_name}!")
        else:
            raise ValueError(f"Unsupported query: {query}!")
        command_list.append(command)
    return command_list

    
def run_experiment(executable, model_list, prop, tool_name, query='aq'):
    results = []
    if query=='aq':
        command_list = generate_aq_command_list(executable, model_list, prop, tool_name, query)
    else:
        command_list = generate_cq_command_list(executable, model_list, prop, tool_name, query)
    runtime_key = query + '_run_time_' + tool_name   
    for switch_model, command in zip(model_list, command_list):                    
        start_time = time.perf_counter()
        data = run_tool_and_parse(command, parse_output, tool_name, query)
        end_time = time.perf_counter()
        data[runtime_key] = end_time - start_time        
        match = re.search(r'switch_model_(\d+)\.prism', os.path.basename(switch_model))
        model_id = None
        if match:
            model_id = int(match.group(1)) 
        else:
            print(f"Warning: Could not extract integer ID from model: {switch_model}")              
        data['max_timesteps'] = model_id
        results.append(data)
    return results

In [143]:
mopmc_gpu_aq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc", "aq")
mopmc_gpu_aq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-GPU Achievability Query Results:")
print(mopmc_gpu_aq_results)

Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_20.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q achievability
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_120.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q achievability
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_200.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.prop

In [102]:
mopmc_cpu_aq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc_cpu", "aq")
mopmc_cpu_aq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-CPU Achievability Query Results:")
print(mopmc_cpu_aq_results)

Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m switch_model_20.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m switch_model_120.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m switch_model_200.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m switch_model_100.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLion

In [144]:
mopmc_gpu_ccq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc", "ccq")
mopmc_gpu_ccq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-GPU Convex Query Results:")
print(mopmc_gpu_ccq_results)

Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_20.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q convex
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_120.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q convex
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_200.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q convex
Ru

In [145]:
mopmc_cpu_ccq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc_cpu", "ccq")
mopmc_cpu_ccq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-CPU Convex Query Results:")
print(mopmc_cpu_ccq_results)

Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_20.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q convex -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_120.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_prop_achievable.props -q convex -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switch_model_200.prism -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/model_size_eval_input/switc

In [ ]:
mopmc_gpu_ucq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc", "ucq")
mopmc_gpu_ucq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-GPU (Unconstrained) Convex Query Results:")
print(mopmc_gpu_ucq_results)

In [108]:
print(mopmc_gpu_ucq_results)

[{'States': 19317, 'Choices': 19377, 'Transitions': 68033, 'ucq_run_time_mopmc': 0.3635582057759166, 'max_timesteps': 20}, {'States': 76937, 'Choices': 77057, 'Transitions': 276673, 'ucq_run_time_mopmc': 0.6320135155692697, 'max_timesteps': 40}, {'States': 172957, 'Choices': 173137, 'Transitions': 626113, 'ucq_run_time_mopmc': 1.3587223328649998, 'max_timesteps': 60}, {'States': 307377, 'Choices': 307617, 'Transitions': 1116353, 'ucq_run_time_mopmc': 2.9655976807698607, 'max_timesteps': 80}, {'States': 480197, 'Choices': 480497, 'Transitions': 1747393, 'ucq_run_time_mopmc': 4.334872174076736, 'max_timesteps': 100}, {'States': 691417, 'Choices': 691777, 'Transitions': 2519233, 'ucq_run_time_mopmc': 9.175834064371884, 'max_timesteps': 120}, {'States': 941037, 'Choices': 941457, 'Transitions': 3431873, 'ucq_run_time_mopmc': 192.1141032492742, 'max_timesteps': 140}, {'States': 1229057, 'Choices': 1229537, 'Transitions': 4485313, 'ucq_run_time_mopmc': 280.88248891662806, 'max_timesteps': 16

In [ ]:
mopmc_cpu_ucq_results= run_experiment(mopmc_executable, switch_model_list, switch_property, "mopmc_cpu", "ucq")
mopmc_cpu_ucq_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nMOPMC-CPU (Unconstrained) Convex Query Results:")
print(mopmc_cpu_ucq_results)

In [120]:
storm_results= run_experiment(storm_executable, switch_model_list, switch_property, "storm")
storm_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nStorm Results:")
print(storm_results)

Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism switch_model_20.prism --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism switch_model_120.prism --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism switch_model_200.prism --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism switch_model_100.prism --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism switch_model_140.prism --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_e

In [121]:
prism_results = run_experiment(prism_executable, switch_model_list, switch_property, "prism")
prism_results.sort(key= lambda item: item['max_timesteps'])
print(f"\nPRISM Results:")
print(prism_results)

Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism switch_model_20.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism switch_model_120.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism switch_model_200.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism switch_model_100.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism switch_model_140.prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/switch_prop_achievable.props
Runnin

In [157]:
combined_results= []
for d0, d1, d2, d3, d4, d5 in zip(storm_results, prism_results, 
                                  mopmc_gpu_aq_results, mopmc_cpu_aq_results,
                                  mopmc_gpu_ccq_results, mopmc_cpu_ccq_results):
    combined_dict = d0.copy()
    combined_dict.update(d1)
    combined_dict.update(d2)
    combined_dict.update(d3)
    combined_dict.update(d4)
    combined_dict.update(d5)
    #combined_dict.pop('tool')
    combined_results.append(combined_dict)

import pandas as pd
combined_results_df = pd.DataFrame(combined_results)
print(f"current column names: {combined_results_df.columns}")
combined_results_df['ccq_run_time_mopmc_per_it'] = combined_results_df['ccq_run_time_mopmc'] / combined_results_df['ccq_loop_count_mopmc']
combined_results_df['ccq_run_time_mopmc_cpu_per_it'] = combined_results_df['ccq_run_time_mopmc_cpu'] / combined_results_df['ccq_loop_count_mopmc_cpu']
new_column_order = ['max_timesteps', 'States', 'Choices', 'Transitions', 
                    'aq_run_time_storm', 'aq_run_time_prism', 
                    'aq_run_time_mopmc', 'aq_run_time_mopmc_cpu', 
                    'ccq_run_time_mopmc', 'ccq_loop_count_mopmc' ,'ccq_run_time_mopmc_per_it', 
                    'ccq_run_time_mopmc_cpu', 'ccq_loop_count_mopmc_cpu', 'ccq_run_time_mopmc_cpu_per_it']
combined_results_df = combined_results_df[new_column_order]

num_columns = ['aq_run_time_storm', 'aq_run_time_prism', 'aq_run_time_mopmc',
       'aq_run_time_mopmc_cpu', 'ccq_run_time_mopmc','ccq_run_time_mopmc_per_it',
       'ccq_run_time_mopmc_cpu', 'ccq_run_time_mopmc_cpu_per_it']
combined_results_df[num_columns] = combined_results_df[num_columns].round(2)
print(combined_results_df)

current column names: Index(['States', 'Choices', 'Transitions', 'aq_run_time_storm',
       'max_timesteps', 'aq_run_time_prism', 'aq_loop_count_mopmc',
       'aq_run_time_mopmc', 'aq_run_time_mopmc_cpu', 'ccq_loop_count_mopmc',
       'ccq_run_time_mopmc', 'ccq_loop_count_mopmc_cpu',
       'ccq_run_time_mopmc_cpu'],
      dtype='object')
   max_timesteps   States  Choices  Transitions  aq_run_time_storm  \
0             20    19317    19377        68033               0.23   
1             40    76937    77057       276673               0.61   
2             60   172957   173137       626113               1.31   
3             80   307377   307617      1116353               2.43   
4            100   480197   480497      1747393               3.91   
5            120   691417   691777      2519233               6.19   
6            140   941037   941457      3431873               8.77   
7            160  1229057  1229537      4485313              12.01   
8            180  1555477 

In [115]:
switch_model_200 = os.path.join(current_folder, 'switch_model_200.prism')
mopmc_ccq_command_switch_model_200 = f"{mopmc_executable} -m {switch_model_200} -p {switch_property} -q convex -c y"
! $mopmc_ccq_command_switch_model_200

--Model Building--
States:	1920297
Choices:	1920897
Transitions:	7014593
____ CUDA INITIALIZING ____
[Main loop] Iteration: 0
[Const. Sat. verification] Verification successful: The point is indeed in all half-spaces.
[Project gradient - interior phase] finds minimum point at iteration: 1 (distance: 1.0071262)
[Minimum norm point optimization] exits for one vertex
[Main loop] Iteration: 1
[Main loop] Iteration: 2
[Const. Sat. verification] Verification successful: The point is indeed in all half-spaces.
[Project gradient - interior phase] finds minimum point at iteration: 1 (distance: 1.004818211)
[Minimum norm point optimization] max margin separation hyperplane computed, terminates at iteration: 1 (distance: 4.658368223e-07)
[Main loop] Iteration: 3
[Main loop] Iteration: 4
[Const. Sat. verification] Verification successful: The point is indeed in all half-spaces.
[Project gradient - interior phase] finds minimum point at iteration: 1 (distance: 1.005607511)
[Minimum norm point optim

In [116]:
switch_model_200 = os.path.join(current_folder, 'switch_model_200.prism')
mopmc_ucq_command_switch_model_200 = f"{mopmc_executable} -m {switch_model_200} -p {switch_property} -q convex -c n"
! $mopmc_ucq_command_switch_model_200

--Model Building--
States:	1920297
Choices:	1920897
Transitions:	7014593
____ CUDA INITIALIZING ____
[Main loop] Iteration: 0
[Project gradient - interior phase] finds minimum point at iteration: 2 (distance: 0)
[Minimum norm point optimization] exits for one vertex
[Main loop] Iteration: 1
[Main loop] Iteration: 2
[Project gradient - interior phase] finds minimum point at iteration: 2 (distance: 9.595566029e-08)
[Minimum norm point optimization] max margin separation hyperplane computed, terminates at iteration: 1 (distance: 2.848689447e-07)
[Main loop] Iteration: 3
[Main loop] Iteration: 4
[Project gradient - interior phase] finds minimum point at iteration: 2 (distance: 6.665356212e-07)
[Minimum norm point optimization] max margin separation hyperplane computed, terminates at iteration: 0 (distance: 6.823016175e-10)
[Main loop] Iteration: 5
[Main loop] Iteration: 6
[Project gradient - interior phase] finds minimum point at iteration: 2 (distance: 6.664827674e-07)
[Minimum norm point

In [59]:
mopmc_gpu_results_sorted = sorted(mopmc_gpu_results, key= lambda item: item['max_timesteps'])
mopmc_cpu_results_sorted = sorted(mopmc_cpu_results, key= lambda item: item['max_timesteps'])
prism_results_sorted = sorted(prism_results, key = lambda item: item['max_timesteps'])
storm_results_sorted = sorted(storm_results, key = lambda item: item['max_timesteps'])
mopmc_gpu_results_sorted

[{'tool': 'mopmc',
  'States': 19317,
  'Transitions': 68033,
  'Choices': 19377,
  'run_time_mopmc': 0.3350753728300333,
  'max_timesteps': 20},
 {'tool': 'mopmc',
  'States': 76937,
  'Transitions': 276673,
  'Choices': 77057,
  'run_time_mopmc': 0.49274621345102787,
  'max_timesteps': 40},
 {'tool': 'mopmc',
  'States': 172957,
  'Transitions': 626113,
  'Choices': 173137,
  'run_time_mopmc': 0.9564441032707691,
  'max_timesteps': 60},
 {'tool': 'mopmc',
  'States': 307377,
  'Transitions': 1116353,
  'Choices': 307617,
  'run_time_mopmc': 1.639373792335391,
  'max_timesteps': 80},
 {'tool': 'mopmc',
  'States': 480197,
  'Transitions': 1747393,
  'Choices': 480497,
  'run_time_mopmc': 2.6883487226441503,
  'max_timesteps': 100},
 {'tool': 'mopmc',
  'States': 691417,
  'Transitions': 2519233,
  'Choices': 691777,
  'run_time_mopmc': 4.185967640951276,
  'max_timesteps': 120},
 {'tool': 'mopmc',
  'States': 941037,
  'Transitions': 3431873,
  'Choices': 941457,
  'run_time_mopmc': 6